In [ ]:
import os
import meshio
import meshplex
import numpy as np
import xarray as xr
import uxarray as uxr
import rioxarray as xrio

# On Docker turn off the warning on PROJ by specifying the PROJ lib path (uncomment the following line)
#os.environ['PROJ_LIB'] = '/opt/conda/envs/gospl/share/proj'

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from scripts import umeshFcts as ufcts

import matplotlib.pyplot as plt
%matplotlib inline

# Create a mesh from scratch

In cases where one to make some generic models, you could build your input file directly in many ways. 

Here is a simple example where we define a flat elevation at 100 m and a simple tectonic uplift (with a linear slope ranging to 5 mm/yr) and export it to a goSPL input file.

In [ ]:
dx = 200 # desired resolution
nx = 501 # desired number of nodes along the x-axis
ny = 501 # desired number of nodes along the y-axis

tmin = 0.
tmax = 0.005

xcoords = np.arange(nx)*float(dx) 
ycoords = np.arange(ny)*float(dx) 
tecx = np.interp(xcoords, [xcoords[0],xcoords[-1]], [tmin,tmax])
tec = np.broadcast_to(tecx, (nx, nx))[:ny,:]


noise = np.random.normal(0, 0.05, tec.shape)
elev = noise+100.

ds = xr.Dataset({
    'elev': xr.DataArray(
                data   = elev,
                dims   = ['y','x'],
                coords = {'x': xcoords,'y': ycoords},
                ),
    'tec': xr.DataArray(
                data   = tec,
                dims   = ['y','x'],
                coords = {'x': xcoords,'y': ycoords},
                )
        }
    )
ds['cellwidth'] = (['y','x'],dx*np.ones( (ny, nx)))
ds.tec.plot()
ds


We then build the unstructured grid for running goSPL

In [ ]:
output_path = "slope_tec" 
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
# Build your planar mesh
ufcts.planarMesh(ds,output_path,fvtk='planar.vtk',fumpas=True,voro=True)


The mesh (`base2D.nc`) is now stored in the output folder (here named `slope`). 

We will open this file and extract the information used in goSPL:

In [ ]:
# Loading the UGRID file
ufile = output_path+'/base2D.nc'
var_name = 'data'
ugrid = uxr.open_grid(ufile) 

# Perform the interpolation (bilinear) 
ufcts.inter2UGRID(ds[['elev','tec']],ugrid,output_path,var_name,type='face',latlon=False)
# ufcts.inter2UGRID(ds[['t']],ugrid,output_path,var_name,type='face',latlon=False)

data_file = [output_path+'/'+var_name+'.nc']
# Get the information related to the mesh: primal and dual mesh
primal_mesh = uxr.open_dataset(ufile, *data_file, use_dual=False)
dual_mesh = uxr.open_dataset(ufile, *data_file, use_dual=True)

# Extract nodes and faces information
ucoords = np.empty((dual_mesh.uxgrid.n_node,3))
ucoords[:,0] = dual_mesh.uxgrid.node_x.values
ucoords[:,1] = dual_mesh.uxgrid.node_y.values
ucoords[:,2] = dual_mesh.uxgrid.node_z.values
ufaces = primal_mesh.uxgrid.node_face_connectivity.values

# Get information about your mesh:
print("Number of nodes: ",len(ucoords)," | number of faces ",len(ufaces))
edge_min = np.round(dual_mesh.uxgrid.edge_node_distances.min().values/1000.+0.,2)
edge_max = np.round(dual_mesh.uxgrid.edge_node_distances.max().values/1000.+0.,2)
edge_mean = np.round(dual_mesh.uxgrid.edge_node_distances.mean().values/1000.+0.,2)
print("edge range (km): min ",edge_min," | max ",edge_max," | mean ",edge_mean)

We now read the created `vtk` file and add the interpolated variables to it:

In [ ]:
mesh = meshio.read(output_path+'/planar.vtk')
vertex = mesh.points
cells = mesh.cells_dict['triangle']
Umesh = meshplex.MeshTri(vertex, cells)
Uarea = Umesh.control_volumes
print('Cell area (km2): ',Uarea.min()*1.e-6,Uarea.max()*1.e-6)

# Define mesh
paleovtk = output_path+"/init.vtk"
vis_mesh = meshio.Mesh(vertex, {"triangle": cells}, 
                       point_data={"elev": primal_mesh.elev.values,
                                   "tec": primal_mesh.tec.values,
                                   },
                       )

# Write it disk
meshio.write(paleovtk, vis_mesh)
print("Writing VTK input file as {}".format(paleovtk))

### Creating goSPL input

We will now create the inputs for goSPL. We first start by creating the input mesh defining our UGRID structure:

In [ ]:
meshname = output_path+"/gospl_mesh"
np.savez_compressed(meshname, v=vertex, c=cells, 
                    z=primal_mesh.elev.data, t=primal_mesh.tec.data
                    )

# Escarpment input

In [ ]:
dx = 1000 # desired resolution
nx = 301 # desired number of nodes along the x-axis
ny = 301 # desired number of nodes along the y-axis

# tmin = 0.
# tmax = 0.005

xcoords = np.arange(nx)*float(dx) 
ycoords = np.arange(ny)*float(dx) 
elev = np.interp(xcoords, [xcoords[0],xcoords[-1]], [0,0])
elev = np.broadcast_to(elev, (nx, nx))[:ny,:]


noise = np.random.normal(0, 0.05, elev.shape)
elev = noise+100.
elev[50:,:] += 900 

ds = xr.Dataset({
    'elev': xr.DataArray(
                data   = elev,
                dims   = ['y','x'],
                coords = {'x': xcoords,'y': ycoords},
                ),
    # 'tec': xr.DataArray(
    #             data   = tec,
    #             dims   = ['y','x'],
    #             coords = {'x': xcoords,'y': ycoords},
    #             )
        }
    )
ds['cellwidth'] = (['y','x'],dx*np.ones( (ny, nx)))
ds.elev.plot()
ds


In [ ]:
output_path = "escarpment" 
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
# Build your planar mesh
ufcts.planarMesh(ds,output_path,fvtk='planar.vtk',fumpas=True,voro=True)


In [ ]:
# Loading the UGRID file
ufile = output_path+'/base2D.nc'
var_name = 'data'
ugrid = uxr.open_grid(ufile) 

# Perform the interpolation (bilinear) 
ufcts.inter2UGRID(ds[['elev']],ugrid,output_path,var_name,type='face',latlon=False)

data_file = [output_path+'/'+var_name+'.nc']
# Get the information related to the mesh: primal and dual mesh
primal_mesh = uxr.open_dataset(ufile, *data_file, use_dual=False)
dual_mesh = uxr.open_dataset(ufile, *data_file, use_dual=True)

# Extract nodes and faces information
ucoords = np.empty((dual_mesh.uxgrid.n_node,3))
ucoords[:,0] = dual_mesh.uxgrid.node_x.values
ucoords[:,1] = dual_mesh.uxgrid.node_y.values
ucoords[:,2] = dual_mesh.uxgrid.node_z.values
ufaces = primal_mesh.uxgrid.node_face_connectivity.values

# Get information about your mesh:
print("Number of nodes: ",len(ucoords)," | number of faces ",len(ufaces))
edge_min = np.round(dual_mesh.uxgrid.edge_node_distances.min().values/1000.+0.,2)
edge_max = np.round(dual_mesh.uxgrid.edge_node_distances.max().values/1000.+0.,2)
edge_mean = np.round(dual_mesh.uxgrid.edge_node_distances.mean().values/1000.+0.,2)
print("edge range (km): min ",edge_min," | max ",edge_max," | mean ",edge_mean)

In [ ]:
mesh = meshio.read(output_path+'/planar.vtk')
vertex = mesh.points
cells = mesh.cells_dict['triangle']
Umesh = meshplex.MeshTri(vertex, cells)
Uarea = Umesh.control_volumes
print('Cell area (km2): ',Uarea.min()*1.e-6,Uarea.max()*1.e-6)

# Define mesh
paleovtk = output_path+"/init.vtk"
vis_mesh = meshio.Mesh(vertex, {"triangle": cells}, 
                       point_data={"elev": primal_mesh.elev.values,
                                   },
                       )

# Write it disk
meshio.write(paleovtk, vis_mesh)
print("Writing VTK input file as {}".format(paleovtk))

In [ ]:
meshname = output_path+"/escarpment2"
np.savez_compressed(meshname, v=vertex, c=cells, 
                    z=primal_mesh.elev.data
                    )

# Create a mesh from a geotiff dataset

We first demonstrate how to create a goSPL inputs from a given geotiff (equivalently one could use a netCDF grid as a starting file).

### Extracting and clipping elevation from geotiff

In [ ]:
output_path = "geotiff" 
if not os.path.exists(output_path):
    os.makedirs(output_path)

xds = xrio.open_rasterio("data/srtm_36_04.tif")
xds

The geotiff contains elevation information in a lon/lat coordinate system:

In [ ]:
plt.figure(figsize=(5,5))
im = xds.sel(band=1).plot(cmap='BrBG', vmin=-0.5, vmax=3000)
im.figure.axes[0].tick_params(axis="both", labelsize=8)
plt.show()

We first reproject the dataset into a UTM grid:

In [ ]:
xds_utm = xds.rio.reproject(xds.rio.estimate_utm_crs())
plt.figure(figsize=(5,5))
im = xds_utm.sel(band=1).plot.imshow(cmap='BrBG', vmin=-0.5, vmax=3000)
im.figure.axes[0].tick_params(axis="both", labelsize=6)
plt.show()

We then crop to a region of interest based on UTM coordinates:

In [ ]:
# Specify the bounding box
geometries = [
    {
        'type': 'Polygon',
        'coordinates': [[
            [400000, 4.5e6],
            [600000, 4.5e6],
            [600000, 4.75e6],
            [400000, 4.75e6]
        ]]
    }
]

# Clip your region
clipped = xds_utm.rio.clip(geometries)
clipped = clipped.sortby(clipped.x)
clipped = clipped.sortby(clipped.y)
clipped

We are now interpolating the data to a specific resolution (here `dx` 500 m):

In [ ]:
dx = 500.
newx = np.arange(401000,600000,dx)
newy = np.arange(4501000,4750000,dx)
newds = clipped.interp(x=newx, y=newy, method="cubic")
elev = newds.sel(band=1).copy()
nds = xr.Dataset({
    'elev': xr.DataArray(
                data   = elev,   
                dims   = ['y','x'],
                coords = {'x': newx, 'y': newy},
                ),
})
nds

### Building a UGRID for goSPL

We will now create an unstructured mesh based on `jigsaw` library. 

First we use the same resolution as our interpolated geotiff dataset (`mesh_res` = `dx`)

In [ ]:
mesh_res = dx

# Build your planar mesh
shape = nds.elev.shape
nds['cellwidth'] = (['y','x'],mesh_res*np.ones(shape))
ufcts.planarMesh(nds,output_path,fvtk='planar.vtk',fumpas=True,voro=True)

The mesh (`base2D.nc`) is now stored in the output folder (here named `geotiff`). 

We will open this file and extract the information used in goSPL:

In [ ]:
# Loading the UGRID file
ufile = output_path+'/base2D.nc'
var_name = 'data'
ugrid = uxr.open_grid(ufile) 

# Perform the interpolation (bilinear) 
ufcts.inter2UGRID(nds[['elev']],ugrid,output_path,var_name,type='face',latlon=False)

data_file = [output_path+'/'+var_name+'.nc']
# Get the information related to the mesh: primal and dual mesh
primal_mesh = uxr.open_dataset(ufile, *data_file, use_dual=False)
dual_mesh = uxr.open_dataset(ufile, *data_file, use_dual=True)

# Extract nodes and faces information
ucoords = np.empty((dual_mesh.uxgrid.n_node,3))
ucoords[:,0] = dual_mesh.uxgrid.node_x.values
ucoords[:,1] = dual_mesh.uxgrid.node_y.values
ucoords[:,2] = dual_mesh.uxgrid.node_z.values
ufaces = primal_mesh.uxgrid.node_face_connectivity.values

# Get information about your mesh:
print("Number of nodes: ",len(ucoords)," | number of faces ",len(ufaces))
edge_min = np.round(dual_mesh.uxgrid.edge_node_distances.min().values/1000.+0.,2)
edge_max = np.round(dual_mesh.uxgrid.edge_node_distances.max().values/1000.+0.,2)
edge_mean = np.round(dual_mesh.uxgrid.edge_node_distances.mean().values/1000.+0.,2)
print("edge range (km): min ",edge_min," | max ",edge_max," | mean ",edge_mean)

We now read the created `vtk` file and add the interpolated variables to it:

In [ ]:
mesh = meshio.read(output_path+'/planar.vtk')
vertex = mesh.points
cells = mesh.cells_dict['triangle']
Umesh = meshplex.MeshTri(vertex, cells)
Uarea = Umesh.control_volumes
print('Cell area (km2): ',Uarea.min()*1.e-6,Uarea.max()*1.e-6)

# Define mesh
paleovtk = output_path+"/init.vtk"
vis_mesh = meshio.Mesh(vertex, {"triangle": cells}, 
                       point_data={"elev": primal_mesh.elev.values,
                                   },
                       )

# Write it disk
meshio.write(paleovtk, vis_mesh)
print("Writing VTK input file as {}".format(paleovtk))

### Creating goSPL input

We will now create the inputs for goSPL. We first start by creating the input mesh defining our UGRID structure:

In [ ]:
meshname = output_path+"/gospl_mesh"
np.savez_compressed(meshname, v=vertex, c=cells, 
                    z=primal_mesh.elev.data
                    )

# Making a variable resolution mesh

Following the approach described above one could create a more complex unstructured mesh with variable cell width based on user defined crietria.

Here we illustrate how this could be done by using the elevation to refine the mesh.

In [ ]:
def weightsElev(ds):

    y = ds.y.values
    x = ds.x.values
    val = ds.elev.copy()
    ds['cellwidth'] = ( ['y','x'], 5000*np.ones((y.size, x.size)))
    ds['cellwidth'] = ds['cellwidth'].where((val<-1000)|(val>0),1500)
    ds['cellwidth'] = ds['cellwidth'].where((val<0)|(val>500),1000)
    ds['cellwidth'] = ds['cellwidth'].where((val<500)|(val>1000),750)
    ds['cellwidth'] = ds['cellwidth'].where((val<1000)|(val>1500),500)
    ds['cellwidth'] = ds['cellwidth'].where(val<1500,250)

    return ds

nds = weightsElev(nds)
plt.figure(figsize=(5,5))
nds.cellwidth.plot()
plt.show()

We now use the same command as before but with variable width:

In [ ]:
mesh_res = dx
output_path = 'geotiff_refined'
if not os.path.exists(output_path):
    os.makedirs(output_path)
    
# Build your planar mesh using the width specified above
ufcts.planarMesh(nds,output_path,fvtk='planar.vtk',fumpas=True,voro=True)

Getting the corresponding mesh information:

In [ ]:
# Loading the UGRID file
ufile = output_path+'/base2D.nc'
var_name = 'data'
ugrid = uxr.open_grid(ufile) 

# Perform the interpolation (bilinear) 
ufcts.inter2UGRID(nds[['elev']],ugrid,output_path,var_name,type='face',latlon=False)

data_file = [output_path+'/'+var_name+'.nc']
# Get the information related to the mesh: primal and dual mesh
primal_mesh = uxr.open_dataset(ufile, *data_file, use_dual=False)
dual_mesh = uxr.open_dataset(ufile, *data_file, use_dual=True)

# Extract nodes and faces information
ucoords = np.empty((dual_mesh.uxgrid.n_node,3))
ucoords[:,0] = dual_mesh.uxgrid.node_x.values
ucoords[:,1] = dual_mesh.uxgrid.node_y.values
ucoords[:,2] = dual_mesh.uxgrid.node_z.values
ufaces = primal_mesh.uxgrid.node_face_connectivity.values

# Get information about your mesh:
print("Number of nodes: ",len(ucoords)," | number of faces ",len(ufaces))
edge_min = np.round(dual_mesh.uxgrid.edge_node_distances.min().values/1000.+0.,2)
edge_max = np.round(dual_mesh.uxgrid.edge_node_distances.max().values/1000.+0.,2)
edge_mean = np.round(dual_mesh.uxgrid.edge_node_distances.mean().values/1000.+0.,2)
print("edge range (km): min ",edge_min," | max ",edge_max," | mean ",edge_mean)

We now read the created `vtk` file and add the interpolated variables to it and save the grid as an input file for goSPL:

In [ ]:
mesh = meshio.read(output_path+'/planar.vtk')
vertex = mesh.points
cells = mesh.cells_dict['triangle']
Umesh = meshplex.MeshTri(vertex, cells)
Uarea = Umesh.control_volumes
print('Cell area (km2): ',Uarea.min()*1.e-6,Uarea.max()*1.e-6)

# Define mesh
paleovtk = output_path+"/init.vtk"
vis_mesh = meshio.Mesh(vertex, {"triangle": cells}, 
                       point_data={"elev": primal_mesh.elev.values,
                                   },
                       )

# Write it disk
meshio.write(paleovtk, vis_mesh)
print("Writing VTK input file as {}".format(paleovtk))

# Save for goSPL inputs
meshname = output_path+"/gospl_mesh"
np.savez_compressed(meshname, v=vertex, c=cells, 
                    z=primal_mesh.elev.data
                    )